In [1]:
wp_text_stage0 = [line.strip().lower() for line in open(r"C:\Users\tenu0.LABRINTH\DataspellProjects\DS-Lab\Week 6\Dataset\War_And_Peace.txt",encoding="utf8")]
print(wp_text_stage0[4000:4010])

['“you must look for husbands for them whether you like it or not....”', '', '“well,” said she, “how’s my cossack?” (márya dmítrievna', 'always called natásha a cossack) and she stroked the child’s arm as', 'she came up fearless and gay to kiss her hand. “i know she’s a scamp', 'of a girl, but i like her.”', '', 'she took a pair of pear-shaped ruby earrings from her huge reticule and,', 'having given them to the rosy natásha, who beamed with the pleasure', 'of her saint’s-day fete, turned away at once and addressed herself to']


In [2]:
wp_text_stage1 = ' '.join(wp_text_stage0)

In [3]:
print(len(wp_text_stage1))
print(wp_text_stage1[40000:40200])

3224829
rove my devotion to you and how i respect your father’s memory, i will do the impossible—your son shall be transferred to the guards. here is my hand on it. are you satisfied?”  “my dear benefactor! t


In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tenu0.LABRINTH\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
from nltk.tokenize import sent_tokenize
wp_text_stage2 = sent_tokenize(wp_text_stage1)

In [6]:
print(len(wp_text_stage2))
print(wp_text_stage2[5000:5010])

26313
['after tea, the company went into the sitting room and princess mary was asked to play on the clavichord.', 'anatole, laughing and in high spirits, came and leaned on his elbows, facing her and beside mademoiselle bourienne.', 'princess mary felt his look with a painfully joyous emotion.', 'her favorite sonata bore her into a most intimately poetic world and the look she felt upon her made that world still more poetic.', 'but anatole’s expression, though his eyes were fixed on her, referred not to her but to the movements of mademoiselle bourienne’s little foot, which he was then touching with his own under the clavichord.', 'mademoiselle bourienne was also looking at princess mary, and in her lovely eyes there was a look of fearful joy and hope that was also new to the princess.', '“how she loves me!” thought princess mary.', '“how happy i am now, and how happy i may be with such a friend and such a husband!', 'husband?', 'can it be possible?” she thought, not daring to look at

In [7]:
fluff = set([line.strip() for line in open("Dataset/stoplist.txt")])

In [8]:
import re
only_alnum = re.compile(r"[^\w]+") ## \w => unicode alphabet
#only_alnum = re.compile(r"[^a-z0-9]") --> This will remove accented characters which are part of many names!

## Replaces one or more occurrence of any characters other unicode alphabets and numbers
def cleanUp(s):
    return re.sub(only_alnum, " ", s).strip()
wp_text_stage3 = [cleanUp(s) for s in wp_text_stage2]
print(wp_text_stage3[4000:4010])

['the whole air reeked with smoke', 'the excited faces of the soldiers were blackened with it', 'some were using their ramrods others putting powder on the touchpans or taking charges from their pouches while others were firing though who they were firing at could not be seen for the smoke which there was no wind to carry away', 'a pleasant humming and whistling of bullets were often heard', 'what is this thought prince andrew approaching the crowd of soldiers', 'it can t be an attack for they are not moving it can t be a square for they are not drawn up for that the commander of the regiment a thin feeble looking old man with a pleasant smile his eyelids drooping more than half over his old eyes giving him a mild expression rode up to bagratión and welcomed him as a host welcomes an honored guest', 'he reported that his regiment had been attacked by french cavalry and that though the attack had been repulsed he had lost more than half his men', 'he said the attack had been repulsed em

In [9]:
def choose_words(s):
    return [w for w in s.split() if len(w) > 2 and w not in fluff]

In [10]:
wp_text_stage4 = [choose_words(sentence) for sentence in wp_text_stage3]
print(wp_text_stage4[4000:4010])

[['the', 'whole', 'air', 'reeked', 'with', 'smoke'], ['the', 'excited', 'faces', 'the', 'soldiers', 'were', 'blackened', 'with'], ['some', 'were', 'using', 'their', 'ramrods', 'others', 'putting', 'powder', 'the', 'touchpans', 'taking', 'charges', 'from', 'their', 'pouches', 'while', 'others', 'were', 'firing', 'though', 'who', 'they', 'were', 'firing', 'could', 'not', 'seen', 'for', 'the', 'smoke', 'which', 'there', 'was', 'wind', 'carry', 'away'], ['pleasant', 'humming', 'and', 'whistling', 'bullets', 'were', 'often', 'heard'], ['what', 'this', 'thought', 'prince', 'andrew', 'approaching', 'the', 'crowd', 'soldiers'], ['can', 'attack', 'for', 'they', 'are', 'not', 'moving', 'can', 'square', 'for', 'they', 'are', 'not', 'drawn', 'for', 'that', 'the', 'commander', 'the', 'regiment', 'thin', 'feeble', 'looking', 'old', 'man', 'with', 'pleasant', 'smile', 'his', 'eyelids', 'drooping', 'more', 'than', 'half', 'over', 'his', 'old', 'eyes', 'giving', 'him', 'mild', 'expression', 'rode', 'ba

In [11]:
print(len(wp_text_stage4))

26313


In [12]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
print(stemmer.stem("running"), stemmer.stem("run"), stemmer.stem("runs"), stemmer.stem("runner"))
print(stemmer.stem("guns"), stemmer.stem("gun"), stemmer.stem("gunned"), stemmer.stem("gunning"))

run run run runner
gun gun gun gun


In [13]:
def stem_list(wordlist):
    return [stemmer.stem(word) for word in wordlist]
for n in range(4000, 4010):
    print(wp_text_stage4[n], stem_list(wp_text_stage4[n]))

['the', 'whole', 'air', 'reeked', 'with', 'smoke'] ['the', 'whole', 'air', 'reek', 'with', 'smoke']
['the', 'excited', 'faces', 'the', 'soldiers', 'were', 'blackened', 'with'] ['the', 'excit', 'face', 'the', 'soldier', 'were', 'blacken', 'with']
['some', 'were', 'using', 'their', 'ramrods', 'others', 'putting', 'powder', 'the', 'touchpans', 'taking', 'charges', 'from', 'their', 'pouches', 'while', 'others', 'were', 'firing', 'though', 'who', 'they', 'were', 'firing', 'could', 'not', 'seen', 'for', 'the', 'smoke', 'which', 'there', 'was', 'wind', 'carry', 'away'] ['some', 'were', 'use', 'their', 'ramrod', 'other', 'put', 'powder', 'the', 'touchpan', 'take', 'charg', 'from', 'their', 'pouch', 'while', 'other', 'were', 'fire', 'though', 'who', 'they', 'were', 'fire', 'could', 'not', 'seen', 'for', 'the', 'smoke', 'which', 'there', 'was', 'wind', 'carri', 'away']
['pleasant', 'humming', 'and', 'whistling', 'bullets', 'were', 'often', 'heard'] ['pleasant', 'hum', 'and', 'whistl', 'bullet', 

In [14]:
wp_text_stage5 = [stem_list(s) for s in wp_text_stage4]
print(wp_text_stage5[4000:4010])

[['the', 'whole', 'air', 'reek', 'with', 'smoke'], ['the', 'excit', 'face', 'the', 'soldier', 'were', 'blacken', 'with'], ['some', 'were', 'use', 'their', 'ramrod', 'other', 'put', 'powder', 'the', 'touchpan', 'take', 'charg', 'from', 'their', 'pouch', 'while', 'other', 'were', 'fire', 'though', 'who', 'they', 'were', 'fire', 'could', 'not', 'seen', 'for', 'the', 'smoke', 'which', 'there', 'was', 'wind', 'carri', 'away'], ['pleasant', 'hum', 'and', 'whistl', 'bullet', 'were', 'often', 'heard'], ['what', 'this', 'thought', 'princ', 'andrew', 'approach', 'the', 'crowd', 'soldier'], ['can', 'attack', 'for', 'they', 'are', 'not', 'move', 'can', 'squar', 'for', 'they', 'are', 'not', 'drawn', 'for', 'that', 'the', 'command', 'the', 'regiment', 'thin', 'feebl', 'look', 'old', 'man', 'with', 'pleasant', 'smile', 'his', 'eyelid', 'droop', 'more', 'than', 'half', 'over', 'his', 'old', 'eye', 'give', 'him', 'mild', 'express', 'rode', 'bagratión', 'and', 'welcom', 'him', 'host', 'welcom', 'honor',

In [15]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [24]:
import gensim
from gensim.models import word2vec
from gensim.models import Phrases
import logging

In [25]:
num_features = 300    # Word vector dimensionality                      
min_word_count = 50   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 6           # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [38]:
wp = word2vec.Word2Vec(wp_text_stage5, workers=num_workers, min_count = min_word_count, window = context, sample = downsampling)

In [39]:
wp.init_sims(replace=True)

C:\Users\tenu0.LABRINTH\AppData\Local\Temp\ipykernel_39648\4092887727.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  wp.init_sims(replace=True)


In [40]:
wp.corpus_count

26313

In [41]:
len(wp.wv.index_to_key)

1183

In [42]:
sorted(list(wp.wv.key_to_index))

['1812',
 'abandon',
 'abl',
 'about',
 'abov',
 'accept',
 'accompani',
 'accord',
 'account',
 'acquaint',
 'across',
 'act',
 'action',
 'activ',
 'ad',
 'address',
 'adjut',
 'admit',
 'advanc',
 'advantag',
 'affair',
 'affect',
 'afraid',
 'after',
 'again',
 'against',
 'age',
 'agit',
 'ago',
 'agre',
 'aid',
 'aim',
 'air',
 'alexand',
 'all',
 'allow',
 'almost',
 'alon',
 'along',
 'alpátych',
 'alreadi',
 'also',
 'alway',
 'amid',
 'among',
 'amus',
 'anatol',
 'and',
 'andrew',
 'angri',
 'angrili',
 'ani',
 'anim',
 'anna',
 'announc',
 'anoth',
 'answer',
 'anxious',
 'anyon',
 'anyth',
 'apart',
 'appar',
 'appear',
 'appoint',
 'approach',
 'approv',
 'are',
 'arm',
 'armi',
 'around',
 'arrang',
 'arriv',
 'artilleri',
 'asham',
 'asid',
 'ask',
 'asleep',
 'assum',
 'assur',
 'attack',
 'attend',
 'attent',
 'attract',
 'austerlitz',
 'austrian',
 'author',
 'avoid',
 'await',
 'away',
 'awkward',
 'back',
 'bad',
 'bagratión',
 'balashëv',
 'bald',
 'ball',
 'barcl

In [43]:
words = ["chair","car","man","woman","clean","close","cloud","coat", "confus","danger","daughter","deal","run","walk","count","father","girl","near","neck","spoke","spoken","stand","show","shown"]

In [44]:
wp.wv.save_word2vec_format('wp.bin')

In [49]:
import numpy as np
X = np.array([wp[w] for w in wp.wv.index_to_key if w in words])
X

TypeError: 'Word2Vec' object is not subscriptable

In [ ]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters = 4)
km.fit(X)
y_kmeans = km.predict(X)

In [ ]:
from sklearn import manifold
lle_data = manifold.LocallyLinearEmbedding(n_neighbors=10, n_components=2).fit_transform(X)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(lle_data[:,0],lle_data[:,1], c =y_kmeans )
for i in range(len(words)-1):
    plt.annotate(words[i], xy = (lle_data[i][0],lle_data[i][1]))
plt.show()